In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
tf.__version__

'2.9.2'

In [ ]:
# 최종 버전
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))#pow function 거듭제곱,cast는 d_model을 float32형으로 변경
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])

    # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
  # query 크기 : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  # key 크기 : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
  # value 크기 : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
  # padding_mask : (batch_size, 1, 1, key의 문장 길이)

  # Q와 K의 곱. 어텐션 스코어 행렬.
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 스케일링
  # dk의 루트값으로 나눠준다.
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 마스킹. 어텐션 스코어 행렬의 마스킹 할 위치에 매우 작은 음수값을 넣는다.
  # 매우 작은 값이므로 소프트맥스 함수를 지나면 행렬의 해당 위치의 값은 0이 된다.
  if mask is not None:
    logits += (mask * -1e9)

  # 소프트맥스 함수는 마지막 차원인 key의 문장 길이 방향으로 수행된다.
  # attention weight : (batch_size, num_heads, query의 문장 길이, key의 문장 길이)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  output = tf.matmul(attention_weights, value)

  return output, attention_weights

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    # d_model을 num_heads로 나눈 값.
    # 논문 기준 : 64
    self.depth = d_model // self.num_heads

    # WQ, WK, WV에 해당하는 밀집층 정의
    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    # WO에 해당하는 밀집층 정의
    self.dense = tf.keras.layers.Dense(units=d_model)

  # num_heads 개수만큼 q, k, v를 split하는 함수
  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # 1. WQ, WK, WV에 해당하는 밀집층 지나기
    # q : (batch_size, query의 문장 길이, d_model)
    # k : (batch_size, key의 문장 길이, d_model)
    # v : (batch_size, value의 문장 길이, d_model)
    # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 2. 헤드 나누기
    # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    # (batch_size, query의 문장 길이, d_model)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    # (batch_size, query의 문장 길이, d_model)
    outputs = self.dense(concat_attention)

    return outputs

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, key의 문장 길이)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 멀티-헤드 어텐션 (첫번째 서브층 / 셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': padding_mask # 패딩 마스크 사용
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 포지션 와이즈 피드 포워드 신경망 (두번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 인코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
# 디코더의 첫번째 서브층(sublayer)에서 미래 토큰을 Mask하는 함수
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x) # 패딩 마스크도 포함
  return tf.maximum(look_ahead_mask, padding_mask)

In [ ]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 멀티-헤드 어텐션 (첫번째 서브층 / 마스크드 셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': look_ahead_mask # 룩어헤드 마스크
      })

  # 잔차 연결과 층 정규화
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 멀티-헤드 어텐션 (두번째 서브층 / 디코더-인코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1, 'key': enc_outputs, 'value': enc_outputs, # Q != K = V
          'mask': padding_mask # 패딩 마스크
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 포지션 와이즈 피드 포워드 신경망 (세번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 디코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):

  # 인코더의 입력
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 디코더의 입력
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더의 패딩 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더의 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 디코더의 패딩 마스크(두번째 서브층)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더의 출력은 enc_outputs. 디코더로 전달된다.
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) # 인코더의 입력은 입력 문장과 패딩 마스크

  # 디코더의 출력은 dec_outputs. 출력층으로 전달된다.
  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 다음 단어 예측을 위한 출력층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [ ]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
import pandas as pd
import urllib.request
import tensorflow_datasets as tfds
import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
file_name='/content/drive/MyDrive/Colab Notebooks/refund_a.csv'
train_data_a = pd.read_csv(file_name)
train_data_a.head()

,a
0,안녕하세요 제가 강원도 순천에 #@소속# 으로 골프여행을 가려 해요
1,그러시군요 제가 무엇을 도와드릴까요
2,비가 올 수도 있다고 써있어서요
3,혹시 환불차 문의하셨나요
4,아니오 그건 아니고 거기 가서도 환불이 되나요


In [ ]:
file_name='/content/drive/MyDrive/Colab Notebooks/refund_q.csv'
train_data = pd.read_csv(file_name)
train_data.head()

,q
0,만나서 반갑습니다 #@소속# #@이름# 상담사 입니다
1,안녕하세요 제가 강원도 순천에 #@소속# 으로 골프여행을 가려 해요
2,그러시군요 제가 무엇을 도와드릴까요
3,비가 올 수도 있다고 써있어서요
4,혹시 환불차 문의하셨나요


In [ ]:
print('챗봇 샘플의 개수 :', len(train_data))

챗봇 샘플의 개수 : 17576


In [ ]:
print(train_data.isnull().sum())

q    0
dtype: int64


In [ ]:
train_data.fillna('', inplace=True)
train_data_a.fillna('', inplace=True)

In [ ]:
print(train_data.isnull().sum())
print(train_data_a.isnull().sum())

q    0
dtype: int64
a    0
dtype: int64


In [ ]:
############################kobert를 사용을 위해서 트랜스포머를 install하였고 실제적인 test는 제작한 transformer를 이용
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 26.4 MB/s 
     |████████████████████████████████| 7.6 MB 59.8 MB/s 
     |████████████████████████████████| 182 kB 64.4 MB/s 


In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
#https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification

In [ ]:

questions = []
for sentence in train_data['q']:
    # 구두점에 대해서 띄어쓰기
    
    sentence = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", " ", sentence)
    sentence=re.sub('동국','소속',sentence)
    sentence=re.sub('정통','이름',sentence)
    sentence = sentence.strip()
    sentence ='[CLS] '+sentence+' [SEP]'
    questions.append(sentence)


questions[:10]

['[CLS] 만나서 반갑습니다   소속    이름  상담사 입니다 [SEP]',
 '[CLS] 안녕하세요 제가 강원도 순천에   소속  으로 골프여행을 가려 해요 [SEP]',
 '[CLS] 그러시군요 제가 무엇을 도와드릴까요 [SEP]',
 '[CLS] 비가 올 수도 있다고 써있어서요 [SEP]',
 '[CLS] 혹시 환불차 문의하셨나요 [SEP]',
 '[CLS] 아니오 그건 아니고 거기 가서도 환불이 되나요 [SEP]',
 '[CLS] 골프장측에서 악천후고지를 하면 전액환불이 가능합니다 [SEP]',
 '[CLS] 그 말은 가 봐야 안 다는 거네요 [SEP]',
 '[CLS] 라운드 중에 중지될 경우엔 규정에 따라 환불이 가능합니다 [SEP]',
 '[CLS] 애매하네요 더 생각해보고 전화 드릴게요 [SEP]']

In [ ]:
answers = []
for sentence in train_data_a['a']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", r" ", sentence)
    sentence=re.sub('동국','소속',sentence)
    sentence=re.sub('정통','이름',sentence)
    sentence ='[CLS] '+sentence+' [SEP]' #버트의 인풋에 맞게 변형
    sentence = sentence.strip()

    answers.append(sentence)

answers[:10]

['[CLS] 안녕하세요 제가 강원도 순천에   소속  으로 골프여행을 가려 해요 [SEP]',
 '[CLS] 그러시군요 제가 무엇을 도와드릴까요 [SEP]',
 '[CLS] 비가 올 수도 있다고 써있어서요 [SEP]',
 '[CLS] 혹시 환불차 문의하셨나요 [SEP]',
 '[CLS] 아니오 그건 아니고 거기 가서도 환불이 되나요 [SEP]',
 '[CLS] 골프장측에서 악천후고지를 하면 전액환불이 가능합니다 [SEP]',
 '[CLS] 그 말은 가 봐야 안 다는 거네요 [SEP]',
 '[CLS] 라운드 중에 중지될 경우엔 규정에 따라 환불이 가능합니다 [SEP]',
 '[CLS] 애매하네요 더 생각해보고 전화 드릴게요 [SEP]',
 '[CLS] 네 알겠습니다 오늘도 행복한 하루 되세요 [SEP]']

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False) 
tokenized_questions = [tokenizer.tokenize (sent) for sent in questions]
tokenized_answers = [tokenizer.tokenize (sent) for sent in answers]


Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
print (questions[3])
print (tokenized_questions[3])
print (answers[3])
print (tokenized_answers[3])

[CLS] 비가 올 수도 있다고 써있어서요 [SEP]
['[CLS]', '비', '##가', '올', '수도', '있다고', '써', '##있', '##어', '##서', '##요', '[SEP]']
[CLS] 혹시 환불차 문의하셨나요 [SEP]
['[CLS]', '혹', '##시', '환', '##불', '##차', '문', '##의', '##하', '##셨', '##나', '##요', '[SEP]']


In [ ]:
listtt=[]
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN=40
#토큰을 숫자 인덱스로 변환
input_ids =[tokenizer.convert_tokens_to_ids(x) for x in tokenized_questions]


#문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids =pad_sequences (input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post" )
input_ids[1]



array([   101,   9521, 118741,  35506,  24982,  48549,   9672,  11287,
         8853,  14279,  12092,   9462,  38631,  10530,  96770,  29805,
         8892,  28396,  29935,  88904,   8843,  26737,   9960,  48549,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

In [ ]:

# 입력 토큰의 최대 시퀀스 길이
MAX_LEN=40
#토큰을 숫자 인덱스로 변환
output_ids =[tokenizer.convert_tokens_to_ids(y) for y in tokenized_answers]


#문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
output_ids =pad_sequences (output_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post" )
output_ids[1]


array([   101,   8924,  30873,  14040,  17360,  48549,   9672,  11287,
         9294, 119137,  10622,   9087,  12638,  15001,  85836, 118671,
        48549,    102,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

In [ ]:
len(questions)

17576

In [ ]:

# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN, END_TOKEN = [101], [102]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size 

In [ ]:
print('시작 토큰 sos 번호  :',START_TOKEN)
print('종료 토큰 eos 번호 :',END_TOKEN)
print('단어 집합의 크기 :',VOCAB_SIZE) ######bert는 서브워드로 한 것의 10배 토큰

시작 토큰 sos 번호  : [101]
종료 토큰 eos 번호 : [102]
단어 집합의 크기 : 119547


In [ ]:
######################토근화 한것으로 #############################
print('질문 데이터의 크기(shape) :', input_ids.shape)
print('답변 데이터의 크기(shape) :', output_ids.shape)

질문 데이터의 크기(shape) : (17576, 40)
답변 데이터의 크기(shape) : (17576, 40)


In [ ]:
# 0번째 샘플을 임의로 출력
print(tokenized_questions[0])
print(tokenized_answers[1])

['[CLS]', '만', '##나', '##서', '반', '##갑', '##습', '##니다', '소속', '이', '##름', '상', '##담', '##사', '입', '##니다', '[SEP]']
['[CLS]', '그', '##러', '##시', '##군', '##요', '제', '##가', '무', '##엇', '##을', '도', '##와', '##드', '##릴', '##까', '##요', '[SEP]']


In [ ]:
print('단어 집합의 크기(Vocab size): {}'.format(VOCAB_SIZE))
print('전체 샘플의 수(Number of samples): {}'.format(len(tokenized_questions)))

단어 집합의 크기(Vocab size): 119547
전체 샘플의 수(Number of samples): 17576


In [ ]:
#########################################################################토큰화#############
# 텐서플로우 dataset을 이용하여 셔플(shuffle)을 수행하되, 배치 크기로 데이터를 묶는다.
####셔플하는 이유는 token의 번호 순서대로 나열 되있는데 이를 섞기 위해서
# 또한 이 과정에서 교사 강요(teacher forcing)을 사용하기 위해서 디코더의 입력과 실제값 시퀀스를 구성한다.
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': input_ids,
        'dec_inputs': output_ids[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs':output_ids[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다.
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# 임의의 샘플에 대해서 [:, :-1]과 [:, 1:]이 어떤 의미를 가지는지 테스트해본다.
print(output_ids[0]) # 기존 샘플
print(output_ids[:1][:, :-1]) # 마지막 패딩 토큰 제거하면서 길이가 39가 된다.
print(output_ids[:1][:, 1:]) # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다. 길이는 역시 39가 된다.

[   101   9521 118741  35506  24982  48549   9672  11287   8853  14279
  12092   9462  38631  10530  96770  29805   8892  28396  29935  88904
   8843  26737   9960  48549    102      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0]
[[   101   9521 118741  35506  24982  48549   9672  11287   8853  14279
   12092   9462  38631  10530  96770  29805   8892  28396  29935  88904
    8843  26737   9960  48549    102      0      0      0      0      0
       0      0      0      0      0      0      0      0      0]]
[[  9521 118741  35506  24982  48549   9672  11287   8853  14279  12092
    9462  38631  10530  96770  29805   8892  28396  29935  88904   8843
   26737   9960  48549    102      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0]]


In [ ]:
tf.keras.backend.clear_session()

# Hyper-parameters
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

(1, 119547, 256)
(1, 119547, 256)


In [ ]:
MAX_LENGTH = 40

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
EPOCHS = 50

model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
275/275 [==============================] - 110s 357ms/step - loss: 4.1302 - accuracy: 0.0239
Epoch 2/50
275/275 [==============================] - 98s 357ms/step - loss: 2.3536 - accuracy: 0.0439
Epoch 3/50
275/275 [==============================] - 98s 358ms/step - loss: 1.5559 - accuracy: 0.1427
Epoch 4/50
275/275 [==============================] - 98s 357ms/step - loss: 1.1316 - accuracy: 0.1894
Epoch 5/50
275/275 [==============================] - 99s 358ms/step - loss: 0.9670 - accuracy: 0.2090
Epoch 6/50
275/275 [==============================] - 99s 358ms/step - loss: 0.8764 - accuracy: 0.2201
Epoch 7/50
275/275 [==============================] - 99s 359ms/step - loss: 0.8155 - accuracy: 0.2283
Epoch 8/50
275/275 [==============================] - 98s 358ms/step - loss: 0.7681 - accuracy: 0.2346
Epoch 9/50
275/275 [==============================] - 98s 357ms/step - loss: 0.7289 - accuracy: 0.2398
Epoch 10/50
275/275 [==============================] - 98s 357ms/step - 

In [ ]:


def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재(마지막) 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 마지막 시점의 예측 단어를 출력에 연결한다.
    # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)


def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
def preprocess_sentence(sentence):
  sentence = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", " ", sentence)
  sentence = sentence.strip()
  return sentence

In [ ]:
output = predict('영화 볼래?')

Input: 영화 볼래?
Output: [CLS] 네 고객님 조금 취소되었습니다


In [ ]:
output=predict('축구 볼래?')

Input: 축구 볼래?
Output: [CLS] 그럼 상품은 제가 받아 보실 수 있는 부분 있습니다


In [ ]:
out=predict('우울해')

Input: 우울해
Output: [CLS] 네 제가 확인하고 안내 도와드리겠습니다


In [ ]:
out=predict('이상하잖아')

Input: 이상하잖아
Output: [CLS] 네 무통장 입금 처리했습니다


In [ ]:
out=predict('오늘 추워')

Input: 오늘 추워
Output: [CLS] 네 이용해 주셔서 감사합니다


In [ ]:
out=predict('내일 할 것 있어?')

Input: 내일 할 것 있어?
Output: [CLS] 네 알겠습니다 취소 할게요


In [ ]:
out=predict('?')

Input: ?
Output: [CLS] 네 고객님 교환 신청을 하셨다고요


In [ ]:
out=predict('!')

Input: !
Output: [CLS] 네 고객님 교환 신청을 하셨다고요


In [ ]:
out=predict('숙제 다 했어?')

Input: 숙제 다 했어?
Output: [CLS] 그럼 반품 비용 차감 후에 환불 진행 예정입니다


In [ ]:
out=predict('환불이요')

Input: 환불이요
Output: [CLS] 네 고객님 환불 신청되셨습니다


In [ ]:
out=predict('환불하고 싶어요')

Input: 환불하고 싶어요
Output: [CLS] 네 환불금 차감을 한번 확인해 보고 환불드릴 수 있습니다


In [ ]:
out=predict('환불 신청했는데 교환으로 변경하고 싶어요')

Input: 환불 신청했는데 교환으로 변경하고 싶어요
Output: [CLS] 네 고객님 언제 환불 기간다고 하셨나요


In [ ]:
out=predict('교환 가능한가요?')

Input: 교환 가능한가요?
Output: [CLS] 상품 불량 및 오배송 등의 경우 교환시 지정 반품 접수 가능합니다


In [ ]:
out=predict('반품 가능한가요?')

Input: 반품 가능한가요?
Output: [CLS] 반품은 상품을 수령하신 날짜를 기준으로 십오일 이내에 가능합니다


In [ ]:
out=predict('제품을 사용했는데 환불 가능한가요?')

Input: 제품을 사용했는데 환불 가능한가요?
Output: [CLS] 네 고객님 환불을 이중으로 구매하신 제품은요


In [ ]:
out=predict('사용했는데 환불 가능한가요?')

Input: 사용했는데 환불 가능한가요?
Output: [CLS] 환불은 결제하신 수단이 어떻게 되실까요


In [ ]:
out=predict('환불 가능한가요?')

Input: 환불 가능한가요?
Output: [CLS] 네 문의하신 내용에 답변을 드리겠습니다


In [ ]:
out=predict('배송지 변경하고 싶어요')

Input: 배송지 변경하고 싶어요
Output: [CLS] 이미 출고된 상태라서 상품 개봉 전에는 교환 가능합니다


In [ ]:
out=predict('환불')

Input: 환불
Output: [CLS] 네 환불 신청하셨습니다


In [ ]:
out=predict('환불하고 싶어요')

Input: 환불하고 싶어요
Output: [CLS] 네 환불금 차감을 한번 확인해 보고 환불드릴 수 있습니다


In [ ]:
out=predict('카드 취소 가능한가요?')

Input: 카드 취소 가능한가요?
Output: [CLS] 네 고객님 어떤 내용인지 알려주세요


In [ ]:
out=predict('연제 오나요')

Input: 연제 오나요
Output: [CLS] 네 해당 부분을 사진으로 확인하고 연락 주세요


In [ ]:
out=predict('택배')

Input: 택배
Output: [CLS] 반품 택배비는 카드결제를 원칙으로 하고 있구요


In [ ]:
out=predict('반품 가능한가요?')

Input: 반품 가능한가요?
Output: [CLS] 반품은 상품을 수령하신 날짜를 기준으로 십오일 이내에 가능합니다


In [ ]:
out=predict('환불하고 싶어요')

Input: 환불하고 싶어요
Output: [CLS] 네 환불금 차감을 한번 확인해 보고 환불드릴 수 있습니다


In [ ]:
out=predict('취소하고 싶어요')

Input: 취소하고 싶어요
Output: [CLS] 마이페이지에서 취소 교환 반품 접수 가능합니다
